# **Chat with any YouTube video for Free**

In [1]:
# Installing the necessary libraries
!pip -qqq install langchain langchain-core langchain-community accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 77.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.8/122.8 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 49.4 MB/s eta 0:00:00


In [8]:
# To ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importing the necessary libraries
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [3]:
# Load the tokenizer and model

model_name = "microsoft/Phi-3-mini-4k-instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/904 [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.8k [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/568 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
# Parameters to control LLM's response generation
generation_args = {
    "max_new_tokens": 512,          # Maximum length of the response
    "return_full_text": False,      # Only return the generated text
}

## **Testing the model**

In [10]:
def query(messages):
  """
  Args:
    messages (list): A list of dictionaries, each with "role" and "content" keys.

  Returns:
    str: The answer from the AI assistant.
  """

  output = pipe(messages, **generation_args)
  return output[0]['generated_text']

Example Usage:

In [11]:
messages = [
    {"role": "system", "content": "You are a helpful digital assistant. Please provide safe, ethical and accurate information to the user."},
    {"role": "user", "content": "What is the capital of Japan?"}
]
result = query(messages)
print(result)

 The capital of Japan is Tokyo. It is the political, economic, and cultural center of the country, housing the Japanese government and the Emperor of Japan. Tokyo became the capital of Japan in 1868, replacing Edo, and has since grown into one of the world's most populous metropolitan areas.


### **Adding a prompt template to modify it's behaviour**

In [12]:
from langchain.prompts import ChatPromptTemplate

template = """
Answer the question based on the context given below. Only answer the question if you're confident. If you do not know the answer, then reply with "I'm sorry, I don't know the answer to this". Your job is to act like a helpful humanoid assistant who provides short but crisp answers.

Context: {context}

Question: {question}
"""

In [13]:
prompt = ChatPromptTemplate.from_template(template)     # Adding the prompt from the template

# Adding the context and the prompt
prompt.format(
    context = "John's favourite colour is red, Jogn married Jane, she hates red but loves yellow!",
    question = "Who is Jane?"
    )

'Human: \nAnswer the question based on the context given below. Only answer the question if you\'re confident. If you do not know the answer, then reply with "I\'m sorry, I don\'t know the answer to this". Your job is to act like a helpful humanoid assistant who provides short but crisp answers.\n\nContext: John\'s favourite colour is red, Jogn married Jane, she hates red but loves yellow!\n\nQuestion: Who is Jane?\n'

## **Transcribing YouTube Videos**

In [14]:
!pip install pytube openai-whisper -qqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 20.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 38.9 MB/s eta 0:00:00


In [15]:
import os
import whisper
from pytube import YouTube

YOUTUBE_VIDEO = "https://www.youtube.com/watch?v=P6FORpg0KVo"

# Check if the transcription file already exists
if not os.path.exists("transcription.txt"):
    # Create a YouTube object
    youtube = YouTube(YOUTUBE_VIDEO)

    # Get the audio stream
    audio = youtube.streams.filter(only_audio=True).first()

    # Load the base Whisper model
    whisper_model = whisper.load_model("base")

    # Define the path to download the audio file
    current_folder = os.getcwd()
    audio_file_path = os.path.join(current_folder, "downloaded_audio.mp4")

    # Download the audio file
    print("Downloading audio...")
    audio.download(output_path=current_folder, filename="downloaded_audio.mp4")
    print(f"Audio downloaded to {audio_file_path}")

    # Transcribe the downloaded audio file
    print("Transcribing audio...")
    transcription = whisper_model.transcribe(audio_file_path, fp16=False)["text"].strip()

    # Write the transcription to a text file
    with open("transcription.txt", "w") as file:
        file.write(transcription)
    print("Transcription completed and saved to transcription.txt")

    # Remove the downloaded audio file
    os.remove(audio_file_path)
    print("Temporary audio file removed")
else:
    print("Transcription file already exists.")

100%|████████████████████████████████████████| 139M/139M [00:00<00:00, 158MiB/s]


Audio downloaded to /content/downloaded_audio.mp4
Transcribing audio...
Transcription completed and saved to transcription.txt
Temporary audio file removed


### **Using the Entire Transcript as Context**

We need to split the transcription as the models cannot take the entire transcript as context directly

In [16]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("transcription.txt")
text_documents = loader.load()        # Contains the entire text transcript

In [17]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Splitting the transcript into chunks of 1000 with an overlap of 50
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
text_splitter.split_documents(text_documents)[:2]       # Checking the first 2 chunks

[Document(page_content="So, I'm from Guatemala. This is a public service announcement that is word Guatemala is. Also, that is not where they keep the prisoners that is called Guantanamo. Not the same place. So, Guatemala is right below Mexico and for the Americans in the audience and let the sink in because it really applies in most ways. For the Americans in the audience, you can think of it as Mexico's Mexico. Just like the US doesn't want illegal immigration from Mexico, Mexico doesn't want illegal immigration from Guatemala. It's a smaller country. It's a poorer country and well, what can I tell you? It has much better Mexican food. Guatemala is a very poor country and a lot of people talk about education as something that brings equality to different social classes. But I always saw it as the opposite, as something that brings inequality. Because what happens in practice is that people who have a lot of money and by themselves be really good education and therefore continue havin

## **Finding the relevant chunks with embeddings**

But how do we find the right chunks of data that contains the relevant information when a user asks a question?

We convert the chunks and the user question to embeddings & find the k-nearest chunk embeddings to the user question's embeddings. Nearby chunks have the highest similarity, which is why we use this approach to find the relevant context for our model.

In [18]:
!pip install sentence-transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.5 MB/s eta 0:00:00


In [19]:
!pip install faiss-cpu
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 11.5 MB/s eta 0:00:00


In [20]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# Read the text from a file
with open("transcription.txt", "r") as f:
    text = f.read()

# Splitting the text into chunks of 1000 with an overlap of 50
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
documents = text_splitter.split_documents(text_documents)

# Load the HuggingFace embeddings
hf_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Create a FAISS vector store from the text chunks
vector_store = FAISS.from_texts(texts=text_splitter.split_text(text), embedding=hf_embeddings)

# Now you can use the vector store for semantic search or other tasks
query = "Where is the speaker from?"
results = vector_store.similarity_search(query, k=2)

print(f"Results for query '{query}':")
for result in results:
    print(result.page_content)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Results for query 'Where is the speaker from?':
a mobile phone or a smartphone in particular. See, building schools all over the world is simple to expensive. On the other hand, most of the world's population already has access to a smartphone and the trend is that that fraction is only going to increase. So we decided at the time that we would make a way to learn foreign languages on a mobile phone that was accessible to everyone. And then we called it Duolingo. Now, in order to truly be accessible to everyone, rich and poor, Duolingo uses a freemium model to support itself. What that means is that you can learn as much as you want without ever having to pay. But if you don't pay, you may have to see an ad at the end of a lesson. Now, if you don't like ads, you can also pay to subscribe to turn off the ads. And it turns out that the vast majority of the revenue for Duolingo comes from people to pay to subscribe to turn off the ads. Now, who are these people who pay to subscribe to tur

See how the chunks related to Guatemala (the place the speaker is from) are retrieved!

## **Chatting with the YouTube video**

In [28]:
# Importing the necessary libraries
from langchain.prompts import ChatPromptTemplate
from langchain.chains import ConversationalRetrievalChain

In [29]:
# Defining the function to interact with the context
def search_context(query):
    results = vector_store.similarity_search(query, k=2)
    return " ".join([result.page_content for result in results])

In [30]:
# Create a chat prompt template
template = """
Answer the question based on the context given below. Only answer the question if you're confident. If you do not know the answer, then reply with "I'm sorry, I don't know the answer to this". Your job is to act like a helpful humanoid assistant who provides short but crisp answers.

Context: {context}

Question: {question}
"""

# Initialize the chat prompt template
chat_prompt = ChatPromptTemplate.from_template(template)

In [32]:
# Define the function to generate a response using the context
def query_with_context(user_input):
    context = search_context(user_input)
    prompt_text = chat_prompt.format(context=context, question=user_input)
    messages = [{"role": "user", "content": prompt_text}]
    output = pipe(messages, **generation_args)
    return output[0]['generated_text']

In [36]:
# Example usage 1
user_input = "Where is the speaker from?"
response = query_with_context(user_input)
print(response)

 The speaker is from Guatemala.


As you can see we get a short but accurate answer.
Let's try another example!

In [37]:
# Example usage 2
user_input = "What is Duolingo's philosophy?"
response = query_with_context(user_input)
print(response)

 Duolingo's philosophy is to make language learning accessible to everyone through a freemium model, allowing users to learn for free with optional ads, and offering a paid subscription to remove ads. They aim to expand this model to other subjects, like math, using mobile phones for widespread education.


This works pretty well with more in-depth questions too!

## **Connecting to Pinecone vector database (Optional)**

In [21]:
!pip install langchain_pinecone -q
!pip install pinecone

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.9/215.9 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 1.5 MB/s eta 0:00:00


In [22]:
%env PINECONE_API_KEY = YOUR_API_KEY
%env PINECONE_ENV = youtube_rag

env: PINECONE_API_KEY=YOUR_API_KEY
env: PINECONE_ENV=youtube_rag


In [23]:
import pinecone

# Create an instance of the Pinecone class
pinecone_client = pinecone.Pinecone(
    api_key = os.getenv("PINECONE_API_KEY"),
    environment = os.getenv("PINECONE_ENV")
)

In [ ]:
from langchain_pinecone import PineconeVectorStore

index_name = "yt-rag"

# To store our chunks of data and the embeddings in Pinecone
pinecone = PineconeVectorStore.from_documents(
    documents,
    hf_embeddings,
    index_name=index_name
)

In [ ]:
# Let's test it
pinecone.similarity_search("Tell me about the ideology behind Duolingo")[:1]

## **Creating a chain with everything connected**

In [25]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

chain = (
    {"context": pinecone.as_retriever(), "question": RunnablePassthrough()}
    | prompt
    | model
)

chain.invoke("What is the philosophy behind Duolingo?")

AttributeError: module 'pinecone' has no attribute 'as_retriever'